In [1]:
from typing import List
import random
import math
from tqdm import tqdm
import numpy as np

In [2]:
class MLP:
    def __init__(self, npl: List[int]): # neurons per layer, d in the slides
        self.npl = list(npl)
        self.L = len(npl)-1  # last layer index
        self.W = []  # weights weight of layer l going from neuron i to neuron j: W[l][i][j]

        for l in range(self.L + 1):
            self.W.append([])
            if (l == 0) : continue # no weights for input layer, to be consistent with indexing (W[1] are weights going to layer 1)

            for i in range(0, self.npl[l-1]+1): # +1 for bias neuron
                self.W[l].append([])
                for j in range(0, self.npl[l]+1):
                    rdm_value = random.random() * 2 - 1  # random value between -1 and 1
                    self.W[l][i].append(0.0 if j == 0 else rdm_value) # no weights going to bias neuron so set to 0.0 if j == 0 corresponds to bias neuron

        self.X = []  # activations of layer l neuron i: X[l][i]
        self.deltas = []  # deltas of layer l neuron i: deltas[l][i]

        for l in range(self.L + 1):
            self.X.append([])
            self.deltas.append([])

            for j in range(0, self.npl[l] + 1):
                self.X[l].append(1.0 if j==0 else 0.0)
                self.deltas[l].append(0.0)

    def _propagate(self, inputs: List[float], is_classification: bool):
        assert(len(inputs) == self.npl[0]) # check if inputs is the same size as nbs of layers

        # update first layer
        for j in range(len(inputs)):
            self.X[0][j + 1] = inputs[j] # update X for input (layer 0) and keep the weight

        # update all layers until output
        for l in range(1, self.L + 1):
            for j in range(1, self.npl[l]+1):
                signal = 0.0
                for i in range(0, self.npl[l-1] + 1):
                    signal += self.W[l][i][j] * self.X[l - 1][i]
                if (is_classification or l != self.L):
                    self.X[l][j] = math.tanh(signal)
                else:
                    self.X[l][j] = signal # use identity method only when regression and updating last layer

    def predict(self, inputs: List[float], is_classification: bool):
        self._propagate(inputs, is_classification)
        return self.X[self.L][1:]
    
    def train(self,
              all_samples_inputs: List[List[float]],
              all_samples_expected_outputs: List[List[float]],
              is_classification: bool,
              num_iter: int,
              alpha: float):
        
        assert(len(all_samples_inputs) == len(all_samples_expected_outputs))
        for _ in tqdm(range(num_iter)):
            k = random.randint(0, len(all_samples_inputs) - 1)
            inputs_k = all_samples_inputs[k]
            labels_k = all_samples_expected_outputs[k]

            self._propagate(inputs_k, is_classification) # mise à jour des sorties de tous les neurons (self.X)

            for j in range(1, self.npl[self.L] + 1): # start with 1 to avoid last layer 1
                self.deltas[self.L][j] = (self.X[self.L][j] - labels_k[j-1])
                if (is_classification) :
                    self.deltas[self.L][j] *= (1.0 - self.X[self.L][j]**2)

            # get the deltas
            for l in reversed(range(2, self.L + 1)):
                for i in range(1, self.npl[l-1] + 1):
                    total = 0.0
                    for j in range(1, self.npl[l] + 1):
                        total += self.W[l][i][j] * self.deltas[l][j]
                    total *= (1.0 - self.X[l-1][i] ** 2)
                    self.deltas[l-1][i] = total

            # update weights
            for l in range(1, self.L + 1):
                for i in range(0, self.npl[l-1] + 1):
                    for j in range(1, self.npl[l] + 1):
                        self.W[l][i][j] -= alpha*self.X[l-1][i]*self.deltas[l][j]

In [7]:
mlp = MLP([2, 3, 1])
print(mlp.X)
print(mlp.W)
print(mlp.deltas)

[[1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0]]
[[], [[0.0, 0.7752035411652591, -0.7216063644188722, -0.6950180558156336], [0.0, -0.38363550086706244, -0.18081422199487762, -0.6628712099644818], [0.0, -0.39070203866735453, -0.1817300188985167, -0.18476663299820162]], [[0.0, 0.3471516069236371], [0.0, 0.1226352783992326], [0.0, 0.5223248264931468], [0.0, -0.0749928541662932]]]
[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0]]


In [118]:
mlp.predict([42.0, 53.0], True)

[-0.9999997717138139]

In [119]:
xor_inputs = [
    [0.0, 0.0],
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
]

xor_expected_ouputs = [
    [-1.0],
    [1.0],
    [1.0],
    [-1.0]
]

for inp in xor_inputs:
    print(mlp.predict(inp, True))
mlp.train(xor_inputs, xor_expected_ouputs, True, 100000, 0.01)

for inp in xor_inputs:
    print(mlp.predict(inp, True))

[-0.7960162180703377]
[-0.9768997558605246]
[-0.9997499759227675]
[-0.9999782765544063]


100%|██████████| 100000/100000 [00:10<00:00, 9575.54it/s]

[-0.9953337365601457]
[0.9944584489935214]
[-0.9999999207494784]
[-0.9978713134533167]


In [111]:
class MLP_matrix:
    def __init__(self, npl: List[int]): # neurons per layer, d in the slides
        self.npl = list(npl)
        self.L = len(npl)-1  # last layer index
        # weights weight of layer l going from neuron i to neuron j: W[l][i][j]
        self.W = []
        self.W.append(np.array([]))  # no weights for input layer, to be consistent with indexing (W[1] are weights going to layer 1)

        for l in range(1, self.L + 1):
            self.W.append(np.random.uniform(-1, 1, (self.npl[l-1]+1, self.npl[l]+1)))  # +1 for bias neuron from layer l-1 to layer l
            self.W[l][ :, 0] = 0.0  # no weights going to bias neuron so set to 0.0, j == 0 corresponds to bias neuron

        self.X = []  # activations of layer l neuron i: X[l][i]
        self.deltas = []  # deltas of layer l neuron i: deltas[l][i]

        for l in range(self.L + 1):
            self.X.append(np.zeros((self.npl[l] + 1, 1)))
            self.X[l][0][0] = 1.0  # bias neuron

            self.deltas.append(np.zeros((self.npl[l] + 1, 1)))

    def _propagate(self, inputs: np.ndarray, is_classification: bool):
        assert(len(inputs) == self.npl[0]) # check if inputs is the same size as nbs of layers

        self.X[0][1:] = inputs # update the X row vector for input layer (keeping bias neuron)

        # update all layers until output
        for l in range(1, self.L + 1):
            signal = self.W[l].T @ self.X[l - 1]  # matrix multiplication

            if (is_classification or l != self.L):
                self.X[l] = np.tanh(signal)
            else:
                self.X[l] = signal # use identity method only when regression and updating last layer

    def predict(self, inputs: np.ndarray, is_classification: bool):
        self._propagate(inputs, is_classification)
        return self.X[self.L][1:]
    
    def train(self,
              X: np.ndarray,
              Y: np.ndarray,
              is_classification: bool,
              num_iter: int,
              alpha: float):
        
        assert(X.shape[1] == Y.shape[1])

        for _ in tqdm(range(num_iter)):
            k = np.random.randint(0, X.shape[1]) # pick a random sample
            X_k = X[:,k].reshape(-1, 1) # get the k-th sample as a column vector
            Y_k = np.ones((Y.shape[0]+1, 1))
            Y_k[1:] = Y[:,k] # add bias neuron at index 0 (easy to handle with matrix operations)

            self._propagate(X_k, is_classification) # mise à jour des sorties de tous les neurons (self.X)

            self.deltas[self.L] = self.X[self.L] - Y_k

            if is_classification:
                self.deltas[self.L] *= (1.0 - self.X[self.L]**2) # square element wise

            # get the deltas
            for l in reversed(range(2, self.L + 1)):
                self.deltas[l-1] = (1.0 - self.X[l-1]**2) * (self.W[l] @ self.deltas[l])

            # update weights
            for l in range(1, self.L + 1):
                self.W[l] -= alpha*self.X[l-1]*self.deltas[l].T

In [112]:
mlp_mat = MLP_matrix([2, 2, 1])
print(mlp_mat.X)
print(mlp_mat.W)
print(mlp_mat.deltas)

[array([[1.],
       [0.],
       [0.]]), array([[1.],
       [0.],
       [0.]]), array([[1.],
       [0.]])]
[array([], dtype=float64), array([[ 0.        , -0.19204588, -0.6227204 ],
       [ 0.        ,  0.87395008, -0.31256436],
       [ 0.        ,  0.17242477,  0.30763333]]), array([[ 0.        , -0.87925168],
       [ 0.        ,  0.30000212],
       [ 0.        ,  0.07860263]])]
[array([[0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.]]), array([[0.],
       [0.]])]


In [113]:
mlp_mat.predict(np.array([[42.0], [53.0]]), True)

array([[0.36067352]])

In [114]:
xor_inputs = np.array([
    [0.0, 1.0, 0.0, 1.0],
    [0.0, 0.0, 1.0, 1.0],
])

xor_expected_ouputs =np.array([
    [-1.0, 1.0, 1.0,-1.0]
])

for col in range(xor_inputs.shape[1]):
    inp = xor_inputs[:, col].reshape(-1, 1)
    print(mlp_mat.predict(inp, True))
mlp_mat.train(xor_inputs, xor_expected_ouputs, True, 100000, 0.01)

for col in range(xor_inputs.shape[1]):
    inp = xor_inputs[:, col].reshape(-1, 1)
    print(mlp_mat.predict(inp, True))

[[-0.10004909]]
[[0.11963283]]
[[-0.0298551]]
[[0.16280006]]


100%|██████████| 100000/100000 [00:03<00:00, 27370.99it/s]

[[-0.97646407]]
[[0.97650081]]
[[0.98403948]]
[[-0.97297899]]


# TODO
- tester avec régression
- implem en C++
- changer vecteur d'état Snake
- implémenter recorder